In [350]:
import time
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import skimage

import cPickle as pickle

In [343]:
import re
import collections
from nltk.corpus import stopwords 
from nltk import word_tokenize
#from nltk.stem import PorterStemmer
#from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [344]:
from image_processing import get_paths

In [345]:
import gensim
from gensim import corpora, models, similarities

# clean text data
* furniture_id: strip & str()
* price: string with $ to float
* features from list to string
* add description & features to one string


In [4]:
sofa_df = pd.read_json('wayfair/sofa.json')

In [5]:
sofa_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 480 entries, 0.0 to 99.0
Data columns (total 15 columns):
category                480 non-null object
colors                  480 non-null object
description             359 non-null object
features                480 non-null object
image_links_all         480 non-null object
image_links_by_color    480 non-null object
manufacturer            480 non-null object
price                   480 non-null object
product_id              480 non-null object
rating_avg              420 non-null float64
rating_count            420 non-null float64
specs                   470 non-null object
title                   480 non-null object
url                     480 non-null object
website                 480 non-null object
dtypes: float64(2), object(13)
memory usage: 60.0+ KB


In [6]:
sofa_df.head().T

,0.0,1.0,10.0,100.0,101.0
category,sofa,sofa,sofa,sofa,sofa
colors,"[Cobblestone, Café, Mocha, Sage, Salsa, Stone]","[Blue Linen / Chevron, Blue Linen / Greek Key,...","[Purple, Red, Brown, Dark Grey, Grey]","[Ash, Graphite, Natural]","[Greenwich Ivory, Greenwich Pecan, Greenwich L..."
description,Add a touch of comfort and style to your home ...,Sleeping on the sofa has never been so chic. S...,This luxurious sofa merging with style and ele...,tion’s mid-century modern design is masterfull...,"Casual and comfy, the Smith Loveseat offers an..."
features,[\nFeatures\n\nHarvest collection\nMaterial: 1...,[\nFeatures\n\nMaterial: 100% Polyester Linen\...,[\nFeatures\n\nCarthusia collection\nRubberwoo...,"[\nFeatures\n\nCovered in durable, easy-to-cle...",[\nFeatures\n\nLoose back and seat cushions\nH...
image_links_all,[https://secure.img2.wfrcdn.com/lf/49/hash/266...,[https://secure.img1.wfrcdn.com/lf/49/hash/338...,[https://secure.img2.wfrcdn.com/lf/49/hash/296...,[https://secure.img1.wfrcdn.com/lf/49/hash/384...,[https://secure.img1.wfrcdn.com/lf/49/hash/308...
image_links_by_color,{u'Stone': [u'https://secure.img2.wfrcdn.com/l...,{u'Blue Linen / Chevron': [u'https://secure.im...,{u'Purple': [u'https://secure.img2.wfrcdn.com/...,{u'Graphite': [u'https://secure.img1.wfrcdn.co...,{u'Greenwich Pecan': [u'https://secure.img1.wf...
manufacturer,Signature Design by Ashley,Mercury Row,Kingstown Home,Carolina Accents,Andover Mills
price,$357.99,$529.99,$829.99,$189.99,$289.99
product_id,GNT3224,MCRR1650,KMDS1431,ENT1245,ANDO1550
rating_avg,4.4,4.1,4.5,4.5,4.5


In [268]:
def clean_text_data(df, sofa=False):
    df['product_id'] = df['product_id'].apply(lambda x: x.strip())
    df['price'] = df['price'].apply(lambda x:float(x.strip('$').replace(',','').split()[0]))
    df['features'] = df['features'].apply(lambda x: '\n'.join(x) if type(x)==list else x)
    df['description'] = df['description'].apply(lambda x: '\n'.join(x) if type(x)==list else x)
    df['description'][df['description'].isnull()] = ''
    df['description_all'] = df['description'] + '\n' + df['features']
    if sofa:
        df = df.set_index('product_id')
    return df 

In [8]:
start = time.time()
sofa_df = clean_text_data(sofa_df)
print time.time() - start

0.0798678398132


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
sofa_df[sofa_df['description_all'] == '\n'].product_id

172     GUS1222
181      LF1136
207     GNT2290
229     BRH3913
320     BRH3828
364     ARA1325
388      LR1360
394     GNT2320
438    CTHM1159
451     GNT2526
Name: product_id, dtype: object

In [10]:
sofa_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 480 entries, 0.0 to 99.0
Data columns (total 16 columns):
category                480 non-null object
colors                  480 non-null object
description             480 non-null object
features                480 non-null object
image_links_all         480 non-null object
image_links_by_color    480 non-null object
manufacturer            480 non-null object
price                   480 non-null float64
product_id              480 non-null object
rating_avg              420 non-null float64
rating_count            420 non-null float64
specs                   470 non-null object
title                   480 non-null object
url                     480 non-null object
website                 480 non-null object
description_all         480 non-null object
dtypes: float64(3), object(13)
memory usage: 63.8+ KB


In [11]:
sofa_df.head().T

,0.0,1.0,10.0,100.0,101.0
category,sofa,sofa,sofa,sofa,sofa
colors,"[Cobblestone, Café, Mocha, Sage, Salsa, Stone]","[Blue Linen / Chevron, Blue Linen / Greek Key,...","[Purple, Red, Brown, Dark Grey, Grey]","[Ash, Graphite, Natural]","[Greenwich Ivory, Greenwich Pecan, Greenwich L..."
description,Add a touch of comfort and style to your home ...,Sleeping on the sofa has never been so chic. S...,This luxurious sofa merging with style and ele...,tion’s mid-century modern design is masterfull...,"Casual and comfy, the Smith Loveseat offers an..."
features,\nFeatures\n\nHarvest collection\nMaterial: 10...,\nFeatures\n\nMaterial: 100% Polyester Linen\n...,\nFeatures\n\nCarthusia collection\nRubberwood...,"\nFeatures\n\nCovered in durable, easy-to-clea...",\nFeatures\n\nLoose back and seat cushions\nHi...
image_links_all,[https://secure.img2.wfrcdn.com/lf/49/hash/266...,[https://secure.img1.wfrcdn.com/lf/49/hash/338...,[https://secure.img2.wfrcdn.com/lf/49/hash/296...,[https://secure.img1.wfrcdn.com/lf/49/hash/384...,[https://secure.img1.wfrcdn.com/lf/49/hash/308...
image_links_by_color,{u'Stone': [u'https://secure.img2.wfrcdn.com/l...,{u'Blue Linen / Chevron': [u'https://secure.im...,{u'Purple': [u'https://secure.img2.wfrcdn.com/...,{u'Graphite': [u'https://secure.img1.wfrcdn.co...,{u'Greenwich Pecan': [u'https://secure.img1.wf...
manufacturer,Signature Design by Ashley,Mercury Row,Kingstown Home,Carolina Accents,Andover Mills
price,357.99,529.99,829.99,189.99,289.99
product_id,GNT3224,MCRR1650,KMDS1431,ENT1245,ANDO1550
rating_avg,4.4,4.1,4.5,4.5,4.5


In [12]:
sofa_df.to_json('wayfair/sofa_clean.json')

# TFIDF: description + features
# tried default tokenizer & my own tokenizer with stemmer - seems to work similarly... can test again on more the whole model.  Can also test with different max_features & max_df.

In [86]:
X = sofa_df['description_all'][sofa_df['description_all'].notnull()]
tfidf = TfidfVectorizer(strip_accents='unicode', stop_words='english', max_df=0.8, 
                        max_features=5000, ngram_range = (1,2))
tfidf_matrix = tfidf.fit_transform(X).todense()

In [87]:
tfidf.get_feature_names()

[u'10',
 u'100',
 u'100 cotton',
 u'100 dacron',
 u'100 poly',
 u'100 polyester',
 u'100 sky',
 u'12',
 u'12 coil',
 u'13',
 u'13 gauge',
 u'16',
 u'18',
 u'1oz',
 u'1oz oz',
 u'20',
 u'20 cut',
 u'21',
 u'21 inches',
 u'22',
 u'22 inches',
 u'25',
 u'25 13',
 u'28',
 u'30',
 u'30 day',
 u'30 days',
 u'32',
 u'33',
 u'34',
 u'34 lbs',
 u'36',
 u'3a',
 u'3a certified',
 u'45',
 u'45 inches',
 u'50',
 u'54',
 u'576',
 u'576 independently',
 u'60',
 u'600',
 u'600 lbs',
 u'600 pounds',
 u'64',
 u'65',
 u'65 warning',
 u'70',
 u'70 plywood',
 u'700',
 u'700 pounds',
 u'72',
 u'75',
 u'75 years',
 u'90',
 u'90 days',
 u'95',
 u'abbott',
 u'abbyson',
 u'abbyson living',
 u'ability',
 u'ac',
 u'ac pacific',
 u'accent',
 u'accent chair',
 u'accent chairs',
 u'accent pillows',
 u'accent toss',
 u'accented',
 u'accented beautifully',
 u'accents',
 u'accentuates',
 u'accessorized',
 u'accessorized matching',
 u'accommodate',
 u'accommodate people',
 u'accommodates',
 u'accommodates size',
 u'acry

In [57]:
def my_tokenizer(doc):
    '''
    INPUT: string
    OUTPUT: list of strings

    Tokenize and stem/lemmatize the document.
    '''   
    
    token = re.findall(r'\w+', doc.lower())
    
    sw = set(stopwords.words('english'))
    token_stop = [word for word in token if word not in sw]
    
    snowball = SnowballStemmer('english')
    sbs_ar = [snowball.stem(word) for word in token_stop]
    return sbs_ar 

In [89]:
tfidf2 = TfidfVectorizer(tokenizer=my_tokenizer, max_features=1000, max_df=0.8, ngram_range = (1,2))
tfidf_matrix2 = tfidf2.fit_transform(X).todense()
tfidf2.get_feature_names()

[u'1',
 u'1 8',
 u'1 year',
 u'100',
 u'100 polyest',
 u'2',
 u'2 assembl',
 u'2 throw',
 u'3',
 u'3 assembl',
 u'34',
 u'34 lbs',
 u'4',
 u'4 5',
 u'5',
 u'5 cushion',
 u'5 inch',
 u'6',
 u'7',
 u'7 8',
 u'8',
 u'8 gaug',
 u'8 hardwood',
 u'8 lbs',
 u'8 oz',
 u'accent',
 u'accent chair',
 u'accent pillow',
 u'accessor',
 u'accessor match',
 u'ad',
 u'add',
 u'addit',
 u'addit comfort',
 u'addit live',
 u'adjust',
 u'adjust headrest',
 u'adorn',
 u'afford',
 u'afford solut',
 u'ahfa',
 u'ahfa standard',
 u'aid',
 u'aid design',
 u'air',
 u'air blown',
 u'allow',
 u'along',
 u'also',
 u'also use',
 u'anchor',
 u'anchor good',
 u'anti',
 u'anti sag',
 u'appeal',
 u'applic',
 u'applic residenti',
 u'approv',
 u'approv wearabl',
 u'area',
 u'area frame',
 u'arm',
 u'arm arm',
 u'arm design',
 u'arm upholsteri',
 u'armrest',
 u'artisan',
 u'artisan craft',
 u'assembl',
 u'assembl requir',
 u'assembl unit',
 u'attach',
 u'attach back',
 u'attract',
 u'avail',
 u'avail multipl',
 u'back',
 u'

# Then cluster with tfidf matrix

In [78]:
km_model = KMeans(n_clusters=10)
km_model.fit(tfidf_matrix)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=10, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [88]:
# max_features=5000
centroids = km_model.cluster_centers_
centroid_top_10_index = [np.argsort(centroids, axis=1)[i][::-1][0:10] for i in xrange(centroids.shape[0])]
feature_names = np.array(tfidf.get_feature_names())
for i in range(len(centroid_top_10_index)):
    print feature_names[centroid_top_10_index[i]]

[u'charcoal blended' u'artisan' u'contemporary living' u'casual living'
 u'compartments' u'completely' u'cause cancer' u'cushions comfortable'
 u'block' u'angle']
[u'corner blocked' u'brown color' u'away' u'areas skillfully' u'burnished'
 u'cushions button' u'cowhide' u'bold' u'blends' u'cushions flipped']
[u'cushion adds' u'cleaners' u'blocking' u'bonded' u'blend upholstery'
 u'brewster' u'blend' u'accessorized matching' u'country' u'chic look']
[u'bags' u'book' u'coordinates' u'70 plywood' u'bonded leather'
 u'comfort cut' u'70' u'clean lines' u'coordinated look' u'charles london']
[u'cotton leather' u'collection year' u'cores constructed' u'charm'
 u'cores' u'commercial residential' u'cleaning' u'combines' u'choose'
 u'care clean']
[u'cloth dipped' u'affordable solution' u'cloth away' u'arms spacious'
 u'assembly breeze' u'assembly arrival' u'artisan' u'comfort converts'
 u'currant twill' u'anti sag']
[u'cushions comfortable' u'contemporary living' u'contained non'
 u'cushion extra'

In [79]:
# max_features=1000
centroids = km_model.cluster_centers_
centroid_top_10_index = [np.argsort(centroids, axis=1)[i][::-1][0:10] for i in xrange(centroids.shape[0])]
feature_names = np.array(tfidf.get_feature_names())
for i in range(len(centroid_top_10_index)):
    print feature_names[centroid_top_10_index[i]]

[u'loveseat' u'cushions' u'sofa' u'linen' u'room' u'seat' u'living' u'wood'
 u'frame' u'collection']
[u'structure' u'heavy' u'double' u'compliment' u'homes' u'wire' u'today'
 u'gauge' u'forms' u'year limited']
[u'using hardwood' u'mind using' u'frames certi' u'giving best'
 u'footprint features' u'hand tailored' u'foam durable' u'best style'
 u'tailored piece' u'manufactured quality']
[u'durability' u'global components' u'states global' u'assembled united'
 u'global' u'quality foam' u'assembled' u'melt fiber' u'states' u'low melt']
[u'support' u'points' u'stress points' u'lumber' u'stress' u'rocker'
 u'minimum' u'promote' u'manufacturing' u'lbs']
[u'north carolina' u'carolina' u'north' u'crafted' u'custom upholstery'
 u'custom' u'cushions' u'quality custom' u'usa features'
 u'collection quality']
[u'wood' u'sofa' u'sleeper' u'warranty' u'frame' u'seating' u'collection'
 u'year' u'leather' u'assembly']
[u'convertible' u'convertible sofa' u'sofa' u'bed' u'faux leather' u'faux'
 u'leather

In [64]:
centroids

array([[  1.50544846e-02,   1.33764255e-02,   2.46414645e-02, ...,
         -9.54097912e-18,  -7.37257477e-18,  -7.37257477e-18],
       [  3.17307205e-02,   1.94945381e-02,   1.53861298e-02, ...,
         -8.67361738e-18,  -6.93889390e-18,  -6.93889390e-18],
       [  2.28476434e-02,   1.45452839e-02,   7.54961971e-03, ...,
          6.01043219e-02,   4.25920605e-02,   4.25920605e-02],
       ..., 
       [  8.64934921e-03,  -5.20417043e-18,   1.61268172e-02, ...,
          3.46944695e-18,   1.30104261e-18,   1.30104261e-18],
       [  3.46944695e-18,   1.73472348e-18,   6.93889390e-18, ...,
          0.00000000e+00,   4.33680869e-19,   4.33680869e-19],
       [  1.15722053e-02,   9.07156046e-03,   3.44180069e-02, ...,
          8.67361738e-19,   8.67361738e-19,   8.67361738e-19]])

In [60]:
text_clustering_result = collections.defaultdict(list)

for i, label in enumerate(km_model.labels_):
    text_clustering_result[label].append(int(X.index[i]))

In [61]:
for i in text_clustering_result:
    print i, text_clustering_result[i]

0 [1, 10, 105, 116, 117, 12, 121, 122, 134, 137, 138, 140, 144, 145, 151, 152, 159, 160, 169, 176, 178, 18, 183, 184, 189, 195, 197, 199, 2, 20, 201, 204, 212, 219, 22, 220, 227, 228, 234, 243, 245, 246, 248, 249, 252, 258, 260, 266, 269, 272, 279, 283, 288, 291, 304, 305, 306, 307, 319, 321, 328, 33, 331, 332, 336, 340, 343, 344, 345, 347, 349, 353, 354, 356, 357, 361, 362, 368, 369, 37, 371, 379, 380, 381, 384, 387, 390, 393, 397, 4, 401, 408, 41, 415, 416, 418, 42, 420, 421, 424, 426, 430, 432, 433, 442, 446, 448, 449, 450, 453, 466, 468, 471, 478, 479, 48, 49, 52, 57, 6, 60, 64, 69, 7, 72, 78, 8, 91, 92, 93, 94, 95]
1 [102, 108, 111, 124, 129, 133, 136, 142, 147, 148, 158, 164, 165, 172, 173, 177, 181, 185, 186, 188, 191, 192, 196, 198, 200, 207, 213, 215, 218, 222, 225, 229, 240, 241, 244, 253, 254, 262, 263, 265, 277, 284, 286, 294, 296, 297, 300, 302, 303, 308, 310, 311, 314, 320, 323, 324, 325, 326, 327, 329, 335, 338, 352, 358, 359, 363, 364, 367, 372, 374, 376, 382, 383, 385,

In [19]:
sofa_df.ix[106].product_id

u'XSQ1448'

# Visualize text clustering result:

In [40]:
category = 'sofa'
paths = get_paths(category)

In [41]:
time.ctime(time.time())

'Mon Sep 28 12:15:24 2015'

In [42]:
for i in text_clustering_result:
    for j in text_clustering_result[i]:
        product_id = sofa_df.ix[j].product_id
        for path in paths:
            if product_id in path:
                image = skimage.io.imread('wayfair/images/' + category + '/' + path)
                new_path = 'wayfair/images/' + category + '/text_clustering/' + str(i) + '/' + path
                skimage.io.imsave(new_path, image)

In [43]:
time.ctime(time.time())

'Mon Sep 28 12:17:05 2015'

# Results: alright I guess?

# Clean all text data & save to new json

In [305]:
sofa_df = pd.read_json('wayfair/sofa.json')
sofa_bed_df = pd.read_json('wayfair/sofa_bed.json')
futon_df = pd.read_json('wayfair/futon.json')
loveseat_df = pd.read_json('wayfair/loveseat.json')
coffee_table_df = pd.read_json('wayfair/coffee_table.json')
desk_df = pd.read_json('wayfair/desk.json')
office_chair_df = pd.read_json('wayfair/office_chair.json')
bookcase_df = pd.read_json('wayfair/bookcase.json')
dining_table_df = pd.read_json('wayfair/dining_table.json')
dining_chair_df = pd.read_json('wayfair/dining_chair.json')
bed_df = pd.read_json('wayfair/bed.json')
nightstand_df = pd.read_json('wayfair/nightstand.json')
dresser_df = pd.read_json('wayfair/dresser.json')

In [306]:
sofa_df = clean_text_data(sofa_df, sofa=True)
sofa_bed_df = clean_text_data(sofa_bed_df, sofa=True)
futon_df = clean_text_data(futon_df, sofa=True)
loveseat_df = clean_text_data(loveseat_df, sofa=True)
coffee_table_df = clean_text_data(coffee_table_df)
desk_df = clean_text_data(desk_df)
office_chair_df = clean_text_data(office_chair_df)
bookcase_df = clean_text_data(bookcase_df)
dining_table_df = clean_text_data(dining_table_df)
dining_chair_df = clean_text_data(dining_chair_df)
bed_df = clean_text_data(bed_df)
nightstand_df = clean_text_data(nightstand_df)
dresser_df = clean_text_data(dresser_df)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [307]:
dresser_df[dresser_df['description_all'] == '\n']

,category,colors,description,features,image_links_all,image_links_by_color,manufacturer,price,product_id,rating_avg,rating_count,specs,title,url,website,description_all


In [308]:
sofa_all_df = sofa_df

# Remove replicates

In [309]:
for i in sofa_bed_df.index:
    if i not in sofa_all_df.index:
        product_df = pd.DataFrame(sofa_bed_df.ix[i]).T
        sofa_all_df = pd.concat([sofa_all_df, product_df], axis=0)
for i in futon_df.index:
    if i not in sofa_all_df.index:
        product_df = pd.DataFrame(futon_df.ix[i]).T
        sofa_all_df = pd.concat([sofa_all_df, product_df], axis=0)
for i in loveseat_df.index:
    if i not in sofa_all_df.index:
        product_df = pd.DataFrame(loveseat_df.ix[i]).T
        sofa_all_df = pd.concat([sofa_all_df, product_df], axis=0)

In [316]:
sofa_all_df = sofa_all_df.reset_index()

In [317]:
sofa_all_df['category'].value_counts()

sofa        480
loveseat    381
sofa_bed    363
futon       260
dtype: int64

In [318]:
sofa_all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1484 entries, 0 to 1483
Data columns (total 16 columns):
product_id              1484 non-null object
category                1484 non-null object
colors                  1484 non-null object
description             1484 non-null object
features                1484 non-null object
image_links_all         1484 non-null object
image_links_by_color    1484 non-null object
manufacturer            1484 non-null object
price                   1484 non-null object
rating_avg              1049 non-null object
rating_count            1049 non-null object
specs                   1414 non-null object
title                   1484 non-null object
url                     1484 non-null object
website                 1484 non-null object
description_all         1484 non-null object
dtypes: object(16)
memory usage: 197.1+ KB


In [312]:
coffee_table_df = coffee_table_df[coffee_table_df.index != 336]
desk_df = desk_df[desk_df.index != 768]
dining_table_df = dining_table_df[dining_table_df.index != 43]
dining_table_df = dining_table_df[dining_table_df.index != 669]
dining_table_df = dining_table_df[dining_table_df.index != 8]
nightstand_df = nightstand_df[nightstand_df.index != 864]

### Still some replicates in dining_chair_df. No replicates in any other dataframes.

In [320]:
sofa_all_df.to_json('wayfair/sofa_all.json')
coffee_table_df.to_json('wayfair/coffee_table.json')
desk_df.to_json('wayfair/desk.json')
office_chair_df.to_json('wayfair/office_chair.json')
bookcase_df.to_json('wayfair/bookcase.json')
dining_table_df.to_json('wayfair/dining_table.json')
dining_chair_df.to_json('wayfair/dining_chair.json')
bed_df.to_json('wayfair/bed.json')
nightstand_df.to_json('wayfair/nightstand.json')
dresser_df.to_json('wayfair/dresser.json')

## Check price

In [338]:
print 'sofa'
print sofa_all_df['price'].shape
print sofa_all_df[sofa_all_df['price']<1000]['price'].shape
print 'coffee_table'
print coffee_table_df['price'].shape
print coffee_table_df[coffee_table_df['price']<500]['price'].shape
print 'desk'
print desk_df['price'].shape
print desk_df[desk_df['price']<500]['price'].shape
print 'office_chair'
print office_chair_df['price'].shape
print office_chair_df[office_chair_df['price']<500]['price'].shape
print 'bookcase'
print bookcase_df['price'].shape
print bookcase_df[bookcase_df['price']<500]['price'].shape
print 'dining_table'
print dining_table_df['price'].shape
print dining_table_df[dining_table_df['price']<700]['price'].shape
print 'dining_chair'
print dining_chair_df['price'].shape
print dining_chair_df[dining_chair_df['price']<500]['price'].shape
print 'bed'
print bed_df['price'].shape
print bed_df[bed_df['price']<700]['price'].shape
print 'nightstand'
print nightstand_df['price'].shape
print nightstand_df[nightstand_df['price']<500]['price'].shape
print 'dresser'
print dresser_df['price'].shape
print dresser_df[dresser_df['price']<700]['price'].shape

sofa
(1484,)
(1211,)
coffee_table
(1007,)
(787,)
desk
(1007,)
(711,)
office_chair
(1008,)
(887,)
bookcase
(1008,)
(812,)
dining_table
(1005,)
(518,)
dining_chair
(1008,)
(940,)
bed
(1008,)
(682,)
nightstand
(1007,)
(840,)
dresser
(1008,)
(689,)


# Combine dining table&chair, office desk&chair:

In [361]:
desk_df = pd.read_json('wayfair/desk.json')
office_chair_df = pd.read_json('wayfair/office_chair.json')

dining_table_df = pd.read_json('wayfair/dining_table.json')
dining_chair_df = pd.read_json('wayfair/dining_chair.json')

In [367]:
office_df = pd.concat([desk_df, office_chair_df], axis=0)

In [368]:
office_df.reset_index(inplace=True)

In [370]:
office_df.drop('index', axis=1, inplace=True)

In [372]:
office_df.head(2)

,category,colors,description,description_all,features,image_links_all,image_links_by_color,manufacturer,price,product_id,rating_avg,rating_count,specs,title,url,website
0,desk,[],Try this minimalist desk to create a workspace...,Try this minimalist desk to create a workspace...,\nFeatures\n\nPull out storage drawer\nStorage...,[https://secure.img1.wfrcdn.com/lf/49/hash/305...,{},Zipcode Design,102.99,ZIPC1663,3.0,28,\nWeights & Dimensions\n\n\n\n Over...,Writing Desk II,http://www.wayfair.com/Writing-Desk-II-ZIPC166...,wayfair
1,desk,[],The Bauman Computer Desk with Hutch is a comfo...,The Bauman Computer Desk with Hutch is a comfo...,\nFeatures\n\nMilled cherry finish\nSlide-out ...,[https://secure.img1.wfrcdn.com/lf/49/hash/309...,{},Three Posts,189.99,THRE1821,4.5,2,\nWeights & Dimensions\n\n\n\n Over...,Bauman Computer Desk with Hutch,http://www.wayfair.com/Bauman-Computer-Desk-wi...,wayfair


In [373]:
dining_df = pd.concat([dining_table_df, dining_chair_df], axis=0)
dining_df.reset_index(inplace=True)
dining_df.drop('index', axis=1, inplace=True)
dining_df.head(2)

,category,colors,description,description_all,features,image_links_all,image_links_by_color,manufacturer,price,product_id,rating_avg,rating_count,specs,title,url,website
0,dining_table,"[Bisque, Dark Brown]",,\n\nFeatures\n\nMestler collection\nMade in Ch...,\nFeatures\n\nMestler collection\nMade in Chin...,[https://secure.img1.wfrcdn.com/lf/49/hash/266...,{u'Dark Brown': [u'https://secure.img1.wfrcdn....,Signature Design by Ashley,420.99,GNT4711,4.5,15,\nWeights & Dimensions\n\n\n\n Over...,Mestler Dining Table,http://www.wayfair.com/Mestler-Dining-Table-D5...,wayfair
1,dining_table,"[Amber Brown, Dark Chocolate, Cherry Walnut, C...",The Reagan Dining Table's classic simplicity m...,The Reagan Dining Table's classic simplicity m...,"\nFeatures\n\nWipe with a clean, damp, lint-fr...",[https://secure.img2.wfrcdn.com/lf/49/hash/250...,{u'Classic Crimson': [u'https://secure.img2.wf...,Birch Lane,1799.00,BL8027,NaN,NaN,\nWeights & Dimensions\nLeaf\n\n\n ...,"Reagan Extending Custom Dining Table, 46 x 76 ...",http://www.wayfair.com/Reagan-46-x-76-Custom-D...,wayfair


In [376]:
office_df.to_json('wayfair/office.json')
dining_df.to_json('wayfair/dining.json')

# Now creating the pipeline for the tfidf of sofa_all & pickle

In [356]:
sofa_all_df = pd.read_json('wayfair/sofa_all.json')
X = sofa_all_df['description_all']
tfidf = TfidfVectorizer(strip_accents='unicode', stop_words='english', max_df=0.8, 
                        max_features=1000, ngram_range = (1,2))
tfidf_matrix = tfidf.fit_transform(X).todense()

In [360]:
start = time.time()
tfidf = TfidfVectorizer(strip_accents='unicode', stop_words='english', max_df=0.8, 
                        max_features=1000, ngram_range = (1,2))
tfidf_matrix = tfidf.fit_transform(X).todense()
print time.time() - start

1.05737304688


In [357]:
tfidf_matrix.shape

(1484, 1000)

In [358]:
with open('wayfair/pickle/sofa_tfidf.pkl', 'w') as f:
    pickle.dump(tfidf, f)
with open('wayfair/pickle/sofa_tfidf_matrix.pkl', 'w') as f:
    pickle.dump(tfidf_matrix, f)

In [359]:
np.savetxt("wayfair/pickle/tfidf_matrix.csv", tfidf_matrix, delimiter=",")